In [1]:
import torchvision
import torch
from torchvision import transforms 
import torch.nn as nn
import torch.nn.functional as F
from kan import KAN
import tqdm

/home/ubuntu/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.is_available()

/home/ubuntu/anaconda3/envs/torch/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    #  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    ]
    )

In [4]:
trainset = torchvision.datasets.MNIST(root='./MNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=500,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(root='./MNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=500,
                                         shuffle=False, num_workers=2)

In [5]:
print(len(trainset),len(testset))
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28*28,64).cuda()
        self.kan = KAN(width=[64,16,10], grid=5, k=3, seed=0,device='cuda:0')
    
    def forward(self,x):
        x = self.linear(x)
        out = self.kan(x)
        return out


net = Net().cuda()

In [6]:
# class MLP(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super().__init__()

#         self.input_fc = nn.Linear(input_dim, 250)
#         self.hidden_fc = nn.Linear(250, 100)
#         self.output_fc = nn.Linear(100, output_dim)

#     def forward(self, x):

#         # x = [batch size, height, width]

#         batch_size = x.shape[0]

#         x = x.view(batch_size, -1)

#         # x = [batch size, height * width]

#         h_1 = F.relu(self.input_fc(x))

#         # h_1 = [batch size, 250]

#         h_2 = F.relu(self.hidden_fc(h_1))

#         # h_2 = [batch size, 100]

#         y_pred = self.output_fc(h_2)

#         # y_pred = [batch size, output dim]

#         return y_pred, h_2

# model= MLP(784,10)

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.002,)
#optimizer = optim.Adam(model.parameters())

for epoch in range(4):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in tqdm.tqdm(enumerate(trainloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # print('predict.size=',pred.size())
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        x = inputs.view(inputs.size(0),-1).cuda()
        outputs = net(x)
        #outputs =model(x)
        loss = criterion(outputs, labels.cuda())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i %100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
    correct = 0
    total = 0
    # net.eval()
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            # calculate outputs by running images through the network
            x = inputs.view(inputs.size(0),-1).cuda()
            #x = inputs.view(inputs.size(0),-1)
            outputs = net(x)
            #outputs =model(x)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.cuda()).sum().item()

        print(f'epoch {epoch} Accuracy of the network on the 10000 test images: {100 * correct // total} %')
    # net.train()
print('Finished Training')

0it [00:00, ?it/s]


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.